## pytorch hello world
* https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

In [89]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [90]:
x.shape, y.shape, x[0].shape

(torch.Size([64, 1000]), torch.Size([64, 10]), torch.Size([1000]))

In [91]:
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

99 67.84065246582031
199 1.6440156698226929
299 0.025622230023145676
399 0.0001580471871420741
499 4.235481299019739e-07


## complex type tensor

- not supported yet : https://github.com/pytorch/pytorch/issues/755

In [92]:
import torch as th
import numpy as np

a = np.array([1+1j,2+2j])
b = np.array([3+3j,4+4j])
ath = th.from_numpy(a)
bth = th.from_numpy(b)
ath_cuda = ath.cuda()
ath_cuda += bth.cuda()
ath = ath_cuda.cpu()
print(ath.numpy())

TypeError: can't convert np.ndarray of type numpy.complex128. The only supported types are: double, float, float16, int64, int32, and uint8.

## imitate rx simulator

In [5]:
import numpy as np
import torch.nn as nn

In [6]:
def gen_rand_complex(low, high):
    r = np.random.uniform(low, high)
    i = np.random.uniform(low, high)
    return r + 1j * i

# UNIT TEST
t_value = gen_rand_complex(-1, 1)
print(t_value)
assert type(t_value) is complex, "error, type should be complex"

(-0.16987431961826283+0.41533789439253366j)


### simple linear model

In [109]:
!pip install torchsummaryX

In [95]:
class Model(nn.Module):
    def __init__(self, in_size, out_size):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_size, 64),
            nn.ReLU(),
            nn.Linear(64, out_size),
            nn.ReLU()
        )
        self.learning_rate = 1e-4
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
    def forward(self, x):
        return self.model(x)
    
m = Model(2,2)
print(m)

loss_fn = nn.MSELoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)


Model(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=2, bias=True)
    (3): ReLU()
  )
)


In [113]:
from torchsummaryX import summary
summary(m, torch.zeros((1, 2)))

                 Kernel Shape Output Shape Params Mult-Adds
Layer                                                      
0_model.Linear_0      [2, 64]      [1, 64]  192.0     128.0
1_model.ReLU_1              -      [1, 64]      -         -
2_model.Linear_2      [64, 2]       [1, 2]  130.0     128.0
3_model.ReLU_3              -       [1, 2]      -         -
-------------------------------------------------------------
                      Totals
Total params           322.0
Trainable params       322.0
Non-trainable params     0.0
Mult-Adds              256.0


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[2, 64]","[1, 64]",192.0,128.0
1_model.ReLU_1,-,"[1, 64]",NaN,NaN
2_model.Linear_2,"[64, 2]","[1, 2]",130.0,128.0
3_model.ReLU_3,-,"[1, 2]",NaN,NaN


In [114]:
####################################################################################
# Simulator
# |- iteration = every clock, receive single signal
####################################################################################

tx_buf = []
rx_buf = []
train_batch_size = 10
iteration = 100

for i in range(iteration):
    tx = gen_rand_complex(-1, 1)
    rx = gen_rand_complex(-1, 1)
    if ((0 < i) and (i % train_batch_size == 0)) or (i == iteration - 1):
        # train
        x = torch.tensor([[x.real, x.imag] for x in rx_buf])
        y = torch.tensor([[x.real, x.imag] for x in tx_buf])
        y_pred = m.forward(x)
        print("\nx={}".format(x))
        print("\ny_pred={}".format(y_pred))
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("train, loss : {}".format(loss))
        tx_buf = []
        rx_buf = []
    else:
        # save on buf
        tx_buf.append(tx)
        rx_buf.append(rx)
        print(".", end="")

..........
x=tensor([[ 0.4426, -0.7361],
        [-0.9835,  0.8398],
        [ 0.0394,  0.5972],
        [ 0.8498, -0.1154],
        [-0.2868,  0.0730],
        [-0.4126, -0.4304],
        [ 0.9563, -0.6813],
        [-0.5588,  0.1995],
        [-0.0563, -0.7030],
        [-0.8544, -0.3600]])

y_pred=tensor([[0.0440, 0.0000],
        [0.0000, 0.1325],
        [0.1757, 0.0593],
        [0.1973, 0.0000],
        [0.0000, 0.0010],
        [0.0000, 0.0000],
        [0.1404, 0.0000],
        [0.0000, 0.0258],
        [0.0000, 0.0000],
        [0.0000, 0.0000]], grad_fn=<ThresholdBackward0>)
train, loss : 0.368866890668869
.........
x=tensor([[-0.4975, -0.3638],
        [ 0.4995,  0.0866],
        [ 0.2356, -0.5844],
        [-0.2909, -0.1573],
        [ 0.9139,  0.3674],
        [-0.0378,  0.5091],
        [ 0.0611,  0.6584],
        [ 0.5415, -0.7570],
        [-0.8469,  0.9909]])

y_pred=tensor([[0.0000, 0.0000],
        [0.1867, 0.0000],
        [0.0068, 0.0000],
        [0.0000, 0.0000]

In [118]:
tmp_x = torch.tensor([[1.,1.]])
m.forward(tmp_x)

tensor([[0.3994, 0.0000]], grad_fn=<ThresholdBackward0>)

### lstm model approach

In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim

In [2]:
target_size = 2
input_size = 2 # one-hot size 
hidden_size = 3 #output from the cell. 바로 결과 예측을 위해서 5로 설정 
num_layers = 1 # one layer rnn이다. (아직 안 다룬 개념)
sequence_length = 4

feature_size = 2
batch_size = 3

In [29]:
class Model(nn.Module): 
    def __init__(self): 
        super(Model,self).__init__() 
        self.rnn=nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True) 
        self.hidden2tag = nn.Linear(hidden_size, target_size)
        self.activate = nn.Tanh()
        
    def forward(self, x, hidden): 
        #input x 를 (batch_size,sequence_length,input_size)로 reshape함 
        #just for make sure이라고 하심. 
        x=x.view(batch_size, sequence_length, input_size) 
        
        #Propagate input through RNN 
        #Input:(batch,seq_len,input_size) 
        out, _ = self.rnn(x, hidden)
        out = self.hidden2tag(out)
        #out = F.log_softmax(out, dim=1)
        out = self.activate(out)
        #for make sure, output이 N * 5 shape을 따르게 하기 위해서 
        out = out.view(-1, target_size) 
        return out
    
    def init_hidden(self): 
        #initialize hidden and cell states 
        return torch.zeros(num_layers, batch_size, hidden_size).to(device)

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
model = Model().to(device) 

loss_fn = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(),lr = 0.1) 

# inputs = inputs.to(device) 
# labels = labels.to(device)

# for epoch in range(100): 
#     optimizer.zero_grad() 
#     loss = 0 
#     hidden = model.init_hidden() 
    
#     # print("predicted string : ",end="")

#     print("inputs : {}, labels : {}".format(inputs, labels))
#     outputs = model(inputs,hidden) 
#     print("outputs : {}".format(outputs))
#     loss = criterion(outputs,labels) 
#     _, idx = outputs.max(1) 
#     print("idx : {}".format(idx))
#     result_str = [idx2char[c] for c in idx.squeeze()] 
#     #print("outputs_sq : {}".format(outputs_sq))
#     print("epoch: %d, loss: %1.3f" % (epoch+1,loss.data)) 
#     print("Predicted string: ", ''.join(result_str)) 
#     loss.backward() 
#     optimizer.step()

In [31]:
7 % 12

7

In [32]:
####################################################################################
# Simulator
# |- iteration = every clock, receive single signal
####################################################################################

tx_buf = []
rx_buf = []
iteration = 100
aggregation_size = batch_size * sequence_length
for i in range(iteration):
    tx = gen_rand_complex(-1, 1)
    rx = gen_rand_complex(-1, 1)
    #if ((0 < i) and (i % aggregation_size == 0)) or (i == iteration - 1):
    if len(tx_buf) >= batch_size * sequence_length: 
        print("iteration : {}, tx_buf len : {}, rx_buf len : {}".format(i, len(tx_buf), len(rx_buf)))
        # train
        x = torch.tensor([[x.real, x.imag] for x in rx_buf])
        y = torch.tensor([[x.real, x.imag] for x in tx_buf])
        #y = torch.tensor([tx_buf[-1].real, tx_buf[-1].imag])
        try:
            # buffer is not enough
            print("trying view x ({}x{}) as size of({},{},{})".format(len(x), len(x[0]), batch_size, sequence_length, feature_size))
            x = x.view(batch_size, sequence_length, feature_size)
        except:
            print(x.shape)
            break
        hidden = model.init_hidden()
        y_pred = model.forward(x, hidden)
        print("\nx={}".format(x))
        print("\ny_pred={}".format(y_pred))
        loss = loss_fn(y_pred[-1], y[-1]) # pick y_pred very last value, y is also : out final result is y_pred[-1]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("train, loss : {}".format(loss))
        tx_buf = []
        rx_buf = []
    else:
        # save on buf
        # TODO : make buf as like queue
        # TODO : as is (chunk style), to be (sliding window style)
        tx_buf.append(tx)
        rx_buf.append(rx)
        print(".", end="")

............iteration : 12, tx_buf len : 12, rx_buf len : 12
trying view x (12x2) as size of(3,4,2)

x=tensor([[[-0.9740, -0.8861],
         [-0.2088, -0.8035],
         [ 0.2793, -0.4247],
         [-0.9584,  0.3434]],

        [[ 0.5459,  0.4039],
         [ 0.2815, -0.5074],
         [-0.6237, -0.7578],
         [ 0.8923, -0.8074]],

        [[-0.4334, -0.4539],
         [ 0.9149,  0.4581],
         [ 0.1481, -0.0930],
         [-0.3368, -0.1613]]])

y_pred=tensor([[-0.2201,  0.3293],
        [-0.0118,  0.3472],
        [ 0.1980,  0.1116],
        [-0.2522, -0.0252],
        [ 0.1400, -0.0583],
        [ 0.1503,  0.3029],
        [ 0.0152,  0.2127],
        [ 0.3194,  0.2847],
        [-0.0903,  0.2036],
        [ 0.2316, -0.0217],
        [ 0.1301,  0.1349],
        [-0.0143,  0.1454]], grad_fn=<ViewBackward>)
train, loss : 0.41503122448921204
............iteration : 25, tx_buf len : 12, rx_buf len : 12
trying view x (12x2) as size of(3,4,2)

x=tensor([[[ 0.7144, -0.4837],
        